# Neural Networks

In [1]:
import optuna
from neural_network import NeuralNetworkHyperparam, FullyConnected, TabularDataset, train, optimizer_dict, activation_dict, loss_criterion_dict
from data import get_data

In [2]:
train_X, train_y, val_X, val_y, test_X, test_y = get_data()
train_dataset = TabularDataset(train_X, train_y)
val_dataset = TabularDataset(val_X, val_y)

In [3]:
print("Optimizers", list(optimizer_dict.keys()))
print("Activation functions", list(activation_dict.keys()))
print("Loss Criteria", list(loss_criterion_dict.keys()))


Optimizers ['optim.Adam', 'optim.SGD', 'optim.Adagrad', 'optim.RMSprop']
Activation functions ['nn.PReLU', 'nn.ReLU', 'nn.LeakyReLU', 'nn.Tanh', 'nn.Sigmoid']
Loss Criteria ['nn.CrossEntropyLoss']


In [4]:
def objective(trial):
    hyperparam = NeuralNetworkHyperparam(
        batch_size=trial.suggest_categorical("batch_size", [1, 4, 16, 64, 256]),
        lr=trial.suggest_loguniform("lr", 1e-4, 1e-1),
        weight_decay=trial.suggest_loguniform("weight_decay", 1e-5, 1e-3),
        optimizer=trial.suggest_categorical("optimizer", list(optimizer_dict.keys())),
        activation=trial.suggest_categorical("activation", list(activation_dict.keys())),
        loss_criterion=trial.suggest_categorical("loss_criterion", list(loss_criterion_dict.keys())),
        num_layers=trial.suggest_categorical("num_layers", [3, 5, 7])
    )
    _, _, val_accuracies = train(
        model=FullyConnected(train_X.shape[1], 3,
                             activation=hyperparam.activation, num_layers=hyperparam.num_layers),
        trainset=train_dataset,
        testset=val_dataset,
        loss_criterion=hyperparam.loss_criterion,
        lr=hyperparam.lr,
        batch_size=hyperparam.batch_size,
        optimizer=hyperparam.optimizer
    )
    return val_accuracies[-1]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-03-08 23:21:11,692] A new study created in memory with name: no-name-64be1127-fc78-4a99-a8cf-ea8118b47f28
[I 2022-03-08 23:21:12,440] Trial 0 finished with value: 79.05882352941177 and parameters: {'batch_size': 64, 'lr': 0.044769565819776765, 'weight_decay': 0.000325914361148099, 'optimizer': 'optim.Adam', 'activation': 'nn.ReLU', 'loss_criterion': 'nn.CrossEntropyLoss', 'num_layers': 5}. Best is trial 0 with value: 79.05882352941177.
[I 2022-03-08 23:21:17,727] Trial 1 finished with value: 79.05882352941177 and parameters: {'batch_size': 4, 'lr': 0.014698543163794339, 'weight_decay': 0.0005606929700251063, 'optimizer': 'optim.SGD', 'activation': 'nn.ReLU', 'loss_criterion': 'nn.CrossEntropyLoss', 'num_layers': 5}. Best is trial 0 with value: 79.05882352941177.
[I 2022-03-08 23:21:19,167] Trial 2 finished with value: 86.82352941176471 and parameters: {'batch_size': 256, 'lr': 0.00032943487409233923, 'weight_decay': 0.0002920635458454676, 'optimizer': 'optim.RMSprop', 'activati

In [6]:
print("Best hyperparameters are:", study.best_params)
print("Best validation accuracy is:", study.best_value)


Best hyperparameters are: {'batch_size': 4, 'lr': 0.00029712481959287323, 'weight_decay': 1.0027579021361967e-05, 'optimizer': 'optim.Adam', 'activation': 'nn.PReLU', 'loss_criterion': 'nn.CrossEntropyLoss', 'num_layers': 7}
Best validation accuracy is: 90.11764705882354
